# tl;dr:
Iteration is fun!  Be lazy and automate all the things!

In [1]:
import pandas as pd

import re
from bs4 import BeautifulSoup as bs
import requests
import nltk

In [2]:
base = 'http://www.springfieldspringfield.co.uk/view_episode_scripts.php?tv-show=game-of-thrones&episode='
seasons = 6
episodes = 10

def all_urls(seasons, episodes):
    all_urls = []
    for s in range(1,seasons+1):
        for e in range(1,episodes+1):
            full_url = base+'s{}e{}'.format(format(s,'02d'),format(e,'02d'))
            #print full_url
            all_urls.append(full_url)
    return all_urls

# scraper

In [3]:
def scrape_got_script(url):
    r = requests.get(url)
    data = r.content
    soup = bs(data, 'html.parser')
 

    text = soup.get_text()
    text = text.split('Game of Thrones Episode Scripts')[1]
    text = text.split('\t\t\t')[1]
    return text

# How much dialogue in each episode?

In [4]:
all_tokens = []
unique_tokens = []
episode_codes = []

for url in all_urls(seasons,episodes):
    text = scrape_got_script(url)
    text = re.sub('\(.*?\)','', text)
    print str(url[-5:]),
    #filename = 'got_script_' + str(url[-5:]) + '.txt'
    #with open(filename, 'w') as f:
    #    f.write(text.encode('utf-8'))
    #print text[:50]
    
    episode_codes.append(str(url[-5:]))
    tokens = nltk.word_tokenize(text)
    all_tokens.append(len(tokens))
    unique_tokens.append(len(set(tokens)))

01e01 01e02 01e03 01e04 01e05 01e06 01e07 01e08 01e09 01e10 02e01 02e02 02e03 02e04 02e05 02e06 02e07 02e08 02e09 02e10 03e01 03e02 03e03 03e04 03e05 03e06 03e07 03e08 03e09 03e10 04e01 04e02 04e03 04e04 04e05 04e06 04e07 04e08 04e09 04e10 05e01 05e02 05e03 05e04 05e05 05e06 05e07 05e08 05e09 05e10 06e01 06e02 06e03 06e04 06e05 06e06 06e07 06e08 06e09 06e10


In [5]:
df = pd.DataFrame()

df['episode'] = episode_codes
df['episode'] = [x.replace('e','')[1:] for x in df['episode']]
df['unique_token_count'] = unique_tokens
df['token_count'] = all_tokens
df['pct_unique_tokens'] = [round(p,3) for p in  df['unique_token_count'] / df['token_count']]

In [8]:
df

,episode,unique_token_count,token_count,pct_unique_tokens
0,101,800,3535,0.226
1,102,1025,4367,0.235
2,103,1120,4621,0.242
3,104,1335,6327,0.211
4,105,1329,6667,0.199
5,106,1016,4518,0.225
6,107,1113,4883,0.228
7,108,957,4127,0.232
8,109,1162,5523,0.210
9,110,1050,4988,0.211


In [37]:
df.to_csv('token_count.csv',index=False)

In [9]:
table = 'token_count'

In [11]:
import sqlite3
conn = sqlite3.connect('got.db')
c = conn.cursor()
df.to_sql(table,conn)
conn.commit()
conn.close()